In [ ]:
!pip install bitsandbytes
!python -m bitsandbytes

import bitsandbytes as bnb


In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [ ]:
prompt = """
<human>: midjourney prompt for a boy running in the snow
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: midjourney prompt for a girl sit on the mountain
<assistant>: What do you want to do on the mountain?
<human>: I want to take a break and enjoy the view.
<assistant>: Alright, take your time.
<human>: 
CPU times: user 8.9 s, sys: 647 ms, total: 9.54 s
Wall time: 13.7 s


# Prep dataset

In [ ]:
import csv

data = [
    {"input": "<human>: How do I calculate the accuracy of a model?",
     "output": "<assistant>: \"Accuracy is the ratio of correctly predicted observations to the total observations. It is calculated as: `Accuracy = (True Positives + True Negatives) / Total Observations`. While accuracy is useful, it can be misleading for imbalanced datasets, so it’s important to consider other metrics such as precision, recall, and F1-score.\""
    },
    {"input": "<human>: What is the purpose of the confusion matrix?",
     "output": "<assistant>: \"A confusion matrix is a table used to describe the performance of a classification model. It shows the actual versus predicted values and includes true positives, false positives, true negatives, and false negatives. It helps to understand the classification model's performance beyond just accuracy.\""
    },
    {"input": "<human>: What is the difference between a box plot and a histogram?",
     "output": "<assistant>: \"A histogram is a graphical representation of the distribution of numerical data, where the data is grouped into bins. A box plot, on the other hand, provides a five-number summary of the data: minimum, first quartile, median, third quartile, and maximum, and it highlights potential outliers.\""
    },
    {"input": "<human>: What is feature scaling, and why is it important?",
     "output": "<assistant>: \"Feature scaling involves normalizing or standardizing the range of independent variables or features of data. It's important because some machine learning algorithms, like gradient descent and K-nearest neighbors, are sensitive to the scale of data and may perform poorly if features have widely different ranges.\""
    },
    {"input": "<human>: What is the difference between overfitting and underfitting?",
     "output": "<assistant>: \"Overfitting occurs when a model performs well on training data but fails to generalize to new, unseen data, as it has learned noise or irrelevant patterns. Underfitting, on the other hand, happens when the model is too simple and cannot capture the underlying structure of the data, leading to poor performance on both training and testing data.\""
    },
    {"input": "<human>: Explain cross-validation and why it is used.",
     "output": "<assistant>: \"Cross-validation is a technique used to assess how well a model generalizes to an independent dataset. It involves splitting the data into several subsets or 'folds'. The model is trained on some folds and validated on the remaining fold. This process is repeated multiple times, and the performance is averaged to get a more accurate estimate of the model's true performance.\""
    },
    {"input": "<human>: What are the differences between bagging and boosting?",
     "output": "<assistant>: \"Bagging and boosting are both ensemble methods in machine learning. Bagging (Bootstrap Aggregating) involves training multiple models in parallel on random subsets of the data and then aggregating their predictions. Boosting trains models sequentially, where each model focuses on correcting the errors of the previous one. Bagging reduces variance, while boosting reduces bias.\""
    },
    {"input": "<human>: What is principal component analysis (PCA) used for?",
     "output": "<assistant>: \"Principal component analysis (PCA) is a dimensionality reduction technique used to reduce the number of features in a dataset while retaining as much variance as possible. It achieves this by transforming the data into a set of orthogonal 'principal components' that capture the most important information.\""
    },
    {"input": "<human>: How does one handle missing data in a dataset?",
     "output": "<assistant>: \"Missing data can be handled in several ways, including removing rows or columns with missing values, imputing missing values using techniques like mean, median, or mode imputation, or using more sophisticated methods like k-nearest neighbors (KNN) imputation or predictive models.\""
    },
    {"input": "<human>: What is a ROC curve, and how is it interpreted?",
     "output": "<assistant>: \"A Receiver Operating Characteristic (ROC) curve is a graphical plot used to assess the performance of a binary classifier. It shows the true positive rate (sensitivity) versus the false positive rate (1-specificity) at various threshold settings. The area under the curve (AUC) measures the model's ability to distinguish between classes, with values closer to 1 indicating better performance.\""
    }
]

# Write to CSV
with open("maximized_datascience_assistant_dataset.csv", "w", newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["input", "output"])
    writer.writeheader()
    writer.writerows(data)

print("Maximized data science assistant dataset created successfully.")


Maximized data science assistant dataset created successfully.


In [ ]:
import pandas as pd

# Load the uploaded CSV file into a DataFrame
data = pd.read_csv("maximized_datascience_assistant_dataset.csv")

# Print the DataFrame to check
print(data)


                                               input  \
0  <human>: How do I calculate the accuracy of a ...   
1  <human>: What is the purpose of the confusion ...   
2  <human>: What is the difference between a box ...   
3  <human>: What is feature scaling, and why is i...   
4  <human>: What is the difference between overfi...   
5  <human>: Explain cross-validation and why it i...   
6  <human>: What are the differences between bagg...   
7  <human>: What is principal component analysis ...   
8  <human>: How does one handle missing data in a...   
9  <human>: What is a ROC curve, and how is it in...   

                                              output  
0  <assistant>: "Accuracy is the ratio of correct...  
1  <assistant>: "A confusion matrix is a table us...  
2  <assistant>: "A histogram is a graphical repre...  
3  <assistant>: "Feature scaling involves normali...  
4  <assistant>: "Overfitting occurs when a model ...  
5  <assistant>: "Cross-validation is a technique ... 

In [ ]:
data

,input,output
0,<human>: How do I calculate the accuracy of a ...,"<assistant>: ""Accuracy is the ratio of correct..."
1,<human>: What is the purpose of the confusion ...,"<assistant>: ""A confusion matrix is a table us..."
2,<human>: What is the difference between a box ...,"<assistant>: ""A histogram is a graphical repre..."
3,"<human>: What is feature scaling, and why is i...","<assistant>: ""Feature scaling involves normali..."
4,<human>: What is the difference between overfi...,"<assistant>: ""Overfitting occurs when a model ..."
5,<human>: Explain cross-validation and why it i...,"<assistant>: ""Cross-validation is a technique ..."
6,<human>: What are the differences between bagg...,"<assistant>: ""Bagging and boosting are both en..."
7,<human>: What is principal component analysis ...,"<assistant>: ""Principal component analysis (PC..."
8,<human>: How does one handle missing data in a...,"<assistant>: ""Missing data can be handled in s..."
9,"<human>: What is a ROC curve, and how is it in...","<assistant>: ""A Receiver Operating Characteris..."


In [ ]:
data["input"][0] # Access the first element of the "input" column

'<human>: How do I calculate the accuracy of a model?'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Initialize the tokenizer and model
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded" # Replace 'model_name' with your specific model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define generation config (adjust as needed)
generation_config = GenerationConfig(
    max_length=50,  # Adjust as needed
    num_return_sequences=1
)

def generate_prompt(data_point):
    prompt = f"""
<human>: {data_point["input"]}
<assistant>:
""".strip()
    return prompt

# Example usage
import pandas as pd

# Sample data (replace with your actual DataFrame loading)
data = pd.DataFrame([
    {"input": "How do I calculate the accuracy of a model?", "output": "Accuracy is the ratio of correctly predicted observations to the total observations..."}
])

data_point = data.iloc[0]  # Get the first data point
prompt = generate_prompt(data_point)

# Generate response
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        **generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 316.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 191.06 MiB is free. Process 11938 has 14.56 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 41.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def generate_prompt(data_point):
    # Extract fields from the data_point dictionary
    return f"""
<human>: {data_point["input"]}
<assistant>: {data_point["output"]}
""".strip()

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

KeyError: 'train'

In [ ]:
data

Dataset({
    features: ['User', 'Prompt', '_id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 289
})

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,4.547500
2,4.680200
3,4.464800
4,4.643400
5,4.624900
6,4.400200
7,4.009800
8,4.232400
9,4.379800
10,4.205100


TrainOutput(global_step=72, training_loss=2.9168864174021616, metrics={'train_runtime': 145.3932, 'train_samples_per_second': 1.988, 'train_steps_per_second': 0.495, 'total_flos': 426667531769856.0, 'train_loss': 2.9168864174021616, 'epoch': 1.0})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "jzdesign/midjourney-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jzdesign/midjourney-falcon-7b/commit/9067710f555feca1d2733005e8a108ff85b62a4c', commit_message='Upload model', commit_description='', oid='9067710f555feca1d2733005e8a108ff85b62a4c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: midjourney prompt for a boy running in the snow
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: midjourney prompt for a boy running in the snow
<assistant>: A boy running in the snow, with a backpack, and a red scarf, by the famous artist, "The Simpsons" --ar 16:9 --w 3000 --h 2000 --no-blur --ar 16:9 --w 3000 --h 2000 --no-blur --ar 16:9 --w 3000 --h 2000 --no-blur --ar 16:9 --w 3000 --h 2000 --no-blur --ar 16:9 --w 3000 --h 2000 --no-blur --ar 16:9 --w 3000 --h 2000 --no-blur --ar 16:9 --w 3000 --h 2000 --no-blur --ar 16
CPU times: user 1min 12s, sys: 0 ns, total: 1min 12s
Wall time: 1min 12s
